In [1]:
# %%timeit
import os
sc_fermi_exec = '/Users/bjm42/source/sc-fermi/upload-sourceforge/frozen-sc-fermi'
print(os.popen(sc_fermi_exec).read())

**************************************************************
  (FROZEN)

   SSSS    CCCC      FFFFFF  EEEEEE   RRRR   MM     MM  IIIII
  SS   S  CC   C     FF      EE      RR   R  MMM   MMM    I
  SS      CC         FF      EE      RR  R   M MM MM M    I
   SSSS   CC     --- FFFFFF  EEEEEE  RRRR    M  MMM  M    I
      SS  CC         FF      EE      R   R   M   M   M    I
  S   SS  CC   C     FF      EE      R   RR  M   M   M    I
   SSSS    CCCC      FF      EEEEEE  R   RR  M   M   M  IIIII


Energies in eV, temperature in Kelvin, DOS in states/unit cell

------
j.buckeridge@ucl.ac.uk 2018
**************************************************************

unitcell.dat found...
(Should be cell for which DOS was determined!)
 
Volume of cell:    1056.622886 A^3
 
Found non-spin polarised system...
 
Number of electrons in system:     544.000000
         Energy gap of system:       5.904000 eV
                 Temperature :    1500.000000 K
     Number of defect species:     2
 
Defects f

In [2]:
import itertools

from py_sc_fermi.defect_species import DefectSpecies
from py_sc_fermi.defect_charge_state import DefectChargeState

def read_input_data(filename, verbose=True, frozen=True, volume=None):
    with open(filename, 'r') as f:
        readin = f.readlines()
        pure_readin = [ l for l in readin if l[0] != '#']
    nspinpol = int(pure_readin.pop(0))
    if verbose:
        if nspinpol == 1:
            print( 'Found non-spin polarised system...' )
        elif nspinpol == 2:
            print( 'Found spin polarised system...' )
        else:
            raise ValueError('spin polarisation specification must be 1 or 2')
    nelect = int(pure_readin.pop(0))
    if verbose:
        print( f'Number of electrons in system: {nelect}')
    egap = float(pure_readin.pop(0))
    if verbose:
        print( f'Energy gap of system: {egap} eV')
        if(egap < 0.0):
            print("You have a negative gap - this is going to get weird...")
    temperature = float(pure_readin.pop(0))
    if verbose:
        print( f'Temperature: {temperature} K')
    ndefects = int(pure_readin.pop(0))
    if verbose:
        print( f'Number of defect species: {ndefects}')
        if ndefects < 0:
            print( '0 defects found...')
    # For each species read in name, number of sites in the unit cell, 
    # charge states, degeneracy and energies
    defect_species = []
    for i in range(ndefects):
        l = pure_readin.pop(0).split()
        name = l[0] 
        ncharge = int(l[1])
        nsites = int(l[2])
        if verbose:
            if ncharge <= 0:
                print(f"ERROR: defect {len[ndefects]+1} has idiotic number of charge states!!")
        charges = []
        energies = []
        degs = []
        for j in range(ncharge):
            l = pure_readin.pop(0).split()
            charges.append(float(l[0]))
            energies.append(float(l[1]))
            degs.append(int(l[2]))
        charge_states  = [ DefectChargeState(c, e, d) for c, e, d in zip( charges, energies, degs ) ]
        defect_species.append( DefectSpecies(name, nsites, charge_states) )
        print(defect_species)
    if frozen == True:
        nfrozen_defects = int(pure_readin.pop(0))
        if verbose:
            print(f'Number of frozen defects: {nfrozen_defects}')
        if nfrozen_defects > 0:
            frozen_defects = {}
            for i in range(nfrozen_defects):
                l = pure_readin.pop(0).split()
                frozen_defects.update({l[0]:l[1]})
        for k,v in frozen_defects.items():
            for i in defect_species:
                if i.name == k:
                    print(v, volume)
                    i.fix_concentration(float(v) / 1e24 * volume) #  
        nfrozen_chgstates = int(pure_readin.pop(0))
        if verbose:
            print(f'Number of frozen charge states: {nfrozen_chgstates}')
        if nfrozen_chgstates > 0:
            frozen_defects = []
            for i in range(nfrozen_chgstates):
                l = pure_readin.pop(0).split()
                frozen_defects.append({'Name': l[0], 'Chg_state': l[1], 'Con': l[2]} )
        defects = {}
        for key, group in itertools.groupby(frozen_defects, lambda item: item["Name"]):
            d = {key: {item["Chg_state"] : item["Con"] for item in group}}
            defects.update(d)
        frozen_defects = []
        for k,v in defects.items():
            charge_states = []
            for l,w in v.items():
                print(l,w)
                chgstate = FrozenDefectChargeState(int(l),float(w) / 1e24 * volume)
                charge_states.append(chgstate)
            if k in [ds.name for ds in defect_species]:
                for i in defect_species:
                    if i.name == k:
                        for cs in charge_states:
                            i.charge_states[cs.charge] = cs
            else:
                defect_species.append(DefectSpecies(k, 1, charge_states))
    return { 'defect_species': defect_species,
         'egap': egap,
         'temperature': temperature,
         'nspinpol': nspinpol,
         'nelect': nelect }

In [3]:
# %%timeit

from pymatgen.io.vasp import Poscar
from py_sc_fermi.inputs import read_dos_data, read_unitcell_data
from py_sc_fermi.defect_system import DefectSystem
from py_sc_fermi.dos import DOS
from py_sc_fermi.defect_charge_state import DefectChargeState, FrozenDefectChargeState
from py_sc_fermi.defect_species import DefectSpecies

# volume = Poscar.from_file('POSCAR').structure.volume
volume = read_unitcell_data(filename='./unitcell.dat')
inputs = read_input_data(filename='./input-fermi-frozen.dat', frozen=True, volume=volume)
dos = read_dos_data('./totdos.dat', nelect=inputs['nelect'], egap=inputs['egap'])
ds = DefectSystem(defect_species=inputs['defect_species'],
                  dos=dos, 
                  temperature=inputs['temperature'], 
                  volume=volume)
print(ds)
ds.get_sc_fermi(verbose=False)


Volume of cell: 1056.622886347127 A^3
Found spin polarised system...
Number of electrons in system: 544
Energy gap of system: 5.904 eV
Temperature: 1500.0 K
Number of defect species: 2
[
v_Li, nsites=3
  q=+0.0, e=3.17135738000001, deg=2
  q=-1.0, e=3.659671988940586, deg=1
]
[
v_Li, nsites=3
  q=+0.0, e=3.17135738000001, deg=2
  q=-1.0, e=3.659671988940586, deg=1
, 
Li_i, nsites=1
  q=+0.0, e=2.9462461100000685, deg=2
  q=+1.0, e=-2.6395306397551033, deg=1
]
Number of frozen defects: 2
3.1710314865459778e+19 1056.622886347127
3.1663613355869524e+19 1056.622886347127
Number of frozen charge states: 34
0 81232108.97428812
1 4858270360477.028
2 88094151494435.89
0 6834829.416309246
-1 0.01009696740488161
-3 120665415533.2387
-4 24693.50540979542
0 7.177914227702731e-18
1 2.1508732100913582e-11
2 6.54131933978193
3 1428870.472723437
4 8928747973.71722
0 2.8798299667086736e-05
1 26.115478827487433
2 77317655178.18588
3 2.00457402440353e-45
0 0.09116187539077451
1 337490.8574190072
2 154812

3.2794199242591873

In [5]:
from scipy.constants import physical_constants
kboltz = physical_constants['Boltzmann constant in eV/K']

In [8]:
[ c*1e24/volume for c in ds.dos.carrier_concentrations(e_fermi=3.3297369374388, kT=1500*kboltz)]

TypeError: ufunc 'true_divide' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [9]:
ds.report()

SC Fermi level :      3.2794199242591873  (eV)

Concentrations:
n (electrons)  : 2250958895875.0205 cm^-3
p (holes)      : 95803922334.83047 cm^-3
v_Li           : 3.1710314865459778e+19 cm^-3
Li_i           : 3.1663613355869524e+19 cm^-3
v_O            : 92952503087021.89 cm^-3
O_i            : 6834829.426406213 cm^-3
v_La           : 120665415533.2387 cm^-3
v_Zr           : 24693.50540979542 cm^-3
Zr_i           : 8930176850.731264 cm^-3
Zr_Li_tet      : 77317655204.3014 cm^-3
Zr_Li          : 1.389734736842328e+16 cm^-3
Zr_La          : 1.0551035417307734e+16 cm^-3
La_Zr          : 261323551395765.72 cm^-3
La_Li          : 2865371068177.6836 cm^-3
Li_La          : 2343728338472214.0 cm^-3
Li_Zr          : 258658142865992.22 cm^-3

Breakdown of concentrations for each defect charge state:
---------------------------------------------------------
v_Li       : Charge Concentration(cm^-3) Total
           :  0.0  2.657908e+10          0.00 
           : -1.0  3.171031e+19          100.0

In [10]:
ds.report(e_fermi=3.3297369374388)

SC Fermi level :      3.3297369374388  (eV)

Concentrations:
n (electrons)  : 3322194821500.9927 cm^-3
p (holes)      : 64912114656.63081 cm^-3
v_Li           : 3.1710314865459778e+19 cm^-3
Li_i           : 3.1663613355869524e+19 cm^-3
v_O            : 92952503087021.89 cm^-3
O_i            : 6834829.426406213 cm^-3
v_La           : 120665415533.2387 cm^-3
v_Zr           : 24693.50540979542 cm^-3
Zr_i           : 8930176850.731264 cm^-3
Zr_Li_tet      : 77317655204.3014 cm^-3
Zr_Li          : 1.389734736842328e+16 cm^-3
Zr_La          : 1.0551035417307734e+16 cm^-3
La_Zr          : 261323551395765.72 cm^-3
La_Li          : 2865371068177.6836 cm^-3
Li_La          : 2343728338472214.0 cm^-3
Li_Zr          : 258658142865992.22 cm^-3

Breakdown of concentrations for each defect charge state:
---------------------------------------------------------
v_Li       : Charge Concentration(cm^-3) Total
           :  0.0  1.800871e+10          0.00 
           : -1.0  3.171031e+19          100.00 


In [ ]:
defspec = ds.defect_species[0]
defspec.get_concentration(e_fermi=3.2794199242591873, temperature=1500.0)

In [ ]:
defspec.get_concentration(e_fermi=3.32973693743879 , temperature=1500.0)

In [ ]:
defspec.charge_state_concentrations(e_fermi=3.32973693743879 , temperature=1500.0)

In [ ]:
defspec.charge_state_concentrations(e_fermi=3.2794199242591873 , temperature=1500.0)

In [ ]:
from py_sc_fermi.defect_charge_state import DefectChargeState, FrozenDefectChargeState
from py_sc_fermi.defect_species import DefectSpecies
inputs = read_input_data(filename='./input-fermi-frozen.dat', frozen=True, volume=volume)

In [ ]:
inputs

In [ ]:
# %%timeit
ds.get_sc_fermi(verbose=False, emin=0.0)

In [ ]:
ds.get_sc_fermi_new(verbose=False, emin=0.0)

In [ ]:
ds.report()

In [ ]:
ds.defect_species

In [ ]:
c_v_Li = ds.defect_species_by_name('v_Li').get_concentration(e_fermi=3.378797377976006, temperature=ds.temperature)

In [ ]:
c_Li_i = ds.defect_species_by_name('Li_i').get_concentration(e_fermi=3.378797377976006, temperature=ds.temperature)

In [ ]:
(c_v_Li - c_Li_i)

In [ ]:
ds.get_sc_fermi()

In [ ]:
ds.get_constrained_sc_fermi(constraint={'v_Li': +1, 'Li_i': -1}, total=-0.3)

In [ ]:
3.1710314865459778e19/1e24*volume

In [ ]:
ds.report()